In [1]:
import ray

ds = ray.data.range(10000)

print(ds.count())
print(ds.take(5))
print(ds.schema())

2024-12-17 11:22:30,433	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-17 11:22:41,569	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-17 11:22:45,534	INFO worker.py:1812 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2024-12-17 11:22:46,482	INFO dataset.py:2631 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2024-12-17 11:22:46,485	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-17_11-22-41_588350_23190/logs/ray-data
2024-12-17 11:22:46,485	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadRange] -> LimitOperator[limit=5]


10000
[dataset]: Run `pip install tqdm` to enable progress reporting.
[{'id': 0}, {'id': 1}, {'id': 2}, {'id': 3}, {'id': 4}]
Column  Type
------  ----
id      int64


In [2]:
# save the dataset to a local file and load it back
ray.data.range(10000).write_csv("local_dir")
ds = ray.data.read_csv("local_dir")
print(ds.count())

2024-12-17 11:23:32,604	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-17_11-22-41_588350_23190/logs/ray-data
2024-12-17 11:23:32,605	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadRange->Write]
2024-12-17 11:23:33,140	INFO datasink.py:103 -- Write operation succeeded. Aggregated write results:
	- num_rows: 10000
	- size_bytes: 80000

2024-12-17 11:23:33,147	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-17_11-22-41_588350_23190/logs/ray-data
2024-12-17 11:23:33,147	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> AggregateNumRows[AggregateNumRows]


10000


In [ ]:
ds1 = ray.data.range(10000)
ds2 = ray.data.range(10000)
ds3 = ds1.union(ds2)
print(ds3.count())

# filter the combined dataset to only the even elements
ds3 = ds3.filter(lambda x: x % 2 ==0)

# sort the filtered dataset
ds3 = ds3.sort()


In [4]:
import numpy as np

ds = ray.data.range(10000).map_batches(lambda batch: np.square(batch).tolist())
ds.take(5)

2024-12-17 11:40:37,842	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-17_11-22-41_588350_23190/logs/ray-data
2024-12-17 11:40:37,843	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadRange->MapBatches(<lambda>)] -> LimitOperator[limit=5]
2024-12-17 11:40:37,871	ERROR streaming_executor_state.py:485 -- An exception was raised from a task of operator "ReadRange->MapBatches(<lambda>)". Dataset execution will now abort. To ignore this exception and continue, set DataContext.max_errored_blocks.


RayTaskError(UserCodeException): [36mray::ReadRange->MapBatches(<lambda>)()[39m (pid=23206, ip=127.0.0.1)
  File "/var/folders/kc/c74q_vx95dx4__wm8dtz6q540000gn/T/ipykernel_23190/845984659.py", line 3, in <lambda>
TypeError: unsupported operand type(s) for *: 'dict' and 'dict'

The above exception was the direct cause of the following exception:

[36mray::ReadRange->MapBatches(<lambda>)()[39m (pid=23206, ip=127.0.0.1)
  File "/Users/zhuoran/.pyenv/versions/3.10.14/envs/DDGRL/lib/python3.10/site-packages/ray/data/_internal/execution/operators/map_operator.py", line 482, in _map_task
    for b_out in map_transformer.apply_transform(iter(blocks), ctx):
  File "/Users/zhuoran/.pyenv/versions/3.10.14/envs/DDGRL/lib/python3.10/site-packages/ray/data/_internal/execution/operators/map_transformer.py", line 392, in __call__
    for data in iter:
  File "/Users/zhuoran/.pyenv/versions/3.10.14/envs/DDGRL/lib/python3.10/site-packages/ray/data/_internal/execution/operators/map_transformer.py", 

In [ ]:
ds.map_batches(MLModel, compute='actors')

cpu_intensive_preprocessing = lambda batch: batch
gpu_intensive_inference = lambda batch: batch

In [ ]:
ds = (ray.data.read_parquet("s3://my_bucket/input_data")
      .window(blocks_per_window=5)
      .map(cpu_intensive_preprocessing)
      .map_batches(gpu_intensive_inference, compute='actors', num_gpus=1)
      .repartition(10)
    )

ds.write_parquet("s3://my_bucket/output_predictions")

In [ ]:
from sklearn import datasets
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split


@ray.remote
class TrainingWorker:
    def __init__(self, alpha: float):
        self._model = SGDClassifier(alpha=alpha)

    def train(self, train_shard: ray.data.Dataset):
        for i, epoch in enumerate(train_shard.iter_epochs()):
            X, Y = zip(*list(epoch.iter_rows()))
            self._model.partial_fit(X, Y, classes=[0, 1])
        
        return self._model

    def test(self, X_test: np.ndarray, Y_test: np.ndarray):
        return self._model.score(X_test, Y_test)



In [ ]:
# distributed part
alpha_vals = [0.00008, 0.00009, 0.0001, 0.00011, 0.00012]

print(f'Starting {len(alpha_vals)} training workers. ')
workers = [TrainingWorker.remote(alpha) for alpha in alpha_vals]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    *datasets.make_classification()
)

train_ds = ray.data.from_items(list(zip(X_train, Y_train)))
shards = (train_ds.repeat(10)
          .random_shuffle_each_window()
          .split(len(workers), locality_hints=workers))

ray.get([worker.train.remote(shard) for worker, shard in zip(workers, shards)])

In [ ]:
# get validation results from each worker
print(ray.get([worker.test.remote(X_test, Y_test) for worker in workers]))

ray.shutdown()

In [ ]:
import ray
from ray.util.dask import enable_dask_on_ray

ray.init()
enable_dask_on_ray()